<a href="https://colab.research.google.com/github/jhmuller/nn_sort/blob/main/sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Teachine a Neural Net to Sort


# Teachine a Neural Net to Sort
Can I make a Neural Net that can sort?
It seemed like an interesting question to me.

The first challenge is how to model the data.
I am sorting integers and the net outputs floats.
I did not think that rounding and comparing
the results to the true value would work well.

So I devised a way to model the problem using 
the moves that bubble sort would make when 
sorting the list.  Each move is encoded
as a vector of length N, where N is the length 
of the input list. All the entries in the vector 
are 0 except the one that tells the position
of the left number to be swapped in this step of Bubble sort.  There are at most N*(N-1)/2 steps.
Once the steps are done, i.e. the list is sorted  subsequent steps are encoded with a 1 in the last position indicating Noop or Nothing to do.

The Net will output probabilities for each vector.
Then I will compare the predicted values to the actual and that will be the loss.

Let's see if it works.

In [1]:
import torch
import numpy as np
from torch import nn
import math
from itertools import permutations
from torch.utils.data import Dataset, DataLoader


## Generating the input data.


In [4]:


def sort_seq(x):
  lst = x.copy()
  N = len(lst)
  swaped = True
  seq = []
  while swaped:
    swaped = False
    for i in range(N-1):
      if lst[i] > lst[i+1]:
        lst[i], lst[i+1] = lst[i+1], lst[i]
        seq.append(i)
        swaped = True
  while len(seq) < N*(N-1)/2:
    seq.append(N-1)
  return lst, seq

def apply_seq(x_in, seq):
  x_out = x_in.copy()
  N = len(x_in)
  for i in seq:
    if i >= N-1:
      continue
    x_out[i], x_out[i+1] = x_out[i+1], x_out[i]
  return x_out

def lst_diff(true, pred):
  diff = 0
  for i, p in enumerate(pred):
    diff += abs(p - true[i])
  return diff

def factorial(n):
  res = n
  for i in range(2, n):
    res = res*i
  return res

def generate_input_data(m=5):
  res = []
  w = int((m*(m-1))/2)
  first = list(range(m))
  iter = permutations(first)
  for i, x in enumerate(iter):
    _, y = sort_seq(list(x))
    mat = np.zeros((w, m))
    for j, p in enumerate(y):
      mat[j, p] = 1
    item = {}
    item["list"] = torch.Tensor(x)
    item["mat"] = mat
    res.append(item)
  print(f" # items: {len(res)}")
  return res


class SortDataset(Dataset):
    """Sort dataset."""

    def __init__(self, data):
      self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return(self.data[idx])

sdata = generate_input_data(m=8)

 # items: 40320


## Split into Train and Test
I put 80% of the permutations in train and the rest in test.  I do this simply by taking the first 80% generated above for train and the rest for test.

Also here I create the train and test dataloaders.

In [5]:
N = len(sdata)
cutoff = int(N*.8)
train = sdata[:cutoff]
test = sdata[cutoff:]
train_dataset = SortDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=4,
                        shuffle=True, num_workers=0)
test_dataset = SortDataset(test)
test_dataloader = DataLoader(test_dataset, batch_size=4,
                        shuffle=True, num_workers=0)

## Define the Net
I will use 2 hidden, linear, layers of size 100.

I reshape the output into a matrix and do softmax on the rows to make probabilities.


In [6]:
import torch.nn.functional as F

class SortNet(nn.Module):
    def __init__(self, lst_len, hidden=100):
        """
        """
        self.lst_len = lst_len
        self.seq_len = int((lst_len * (lst_len-1))/2)
        out_len = (lst_len) * self.seq_len        
        print(lst_len, out_len)
        super().__init__()
        self.linear1 = nn.Linear(lst_len, hidden)        
        self.linear2 = nn.Linear(hidden, out_len)

    def forward(self, x):
            """
            """
            h_relu = F.relu(self.linear1(x))
            y = torch.sigmoid(self.linear2(h_relu))
            y_pred = y.view((y.shape[0], self.seq_len , self.lst_len))
            y_pred = torch.softmax(y_pred, dim=2)
            return y_pred

## Train loop
Nothing too unusual here, mostly a standard torch train loop.

I do have to convert the input to tensors and to float32.

In [ ]:
print_freq = 20
model = SortNet(8)
loss_fn = nn.MSELoss() # nn.L1Loss() # 
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
for ei in range(121):
  model.train()
  for bi, sample in enumerate(train_dataloader):
    X = torch.Tensor(sample["list"])
    X = X.to(torch.float32)
    y = torch.Tensor(sample["mat"])
    y = y.to(torch.float32)
    y_pred = model(X)          

    optimizer.zero_grad()       
    loss = loss_fn(y, y_pred)  
    loss.backward()        
    optimizer.step()
  if ei % print_freq == 0: 
    print(f"epoch {ei} batch {bi} loss {loss}")

8 224
epoch 0 batch 8063 loss 0.08435816317796707


In [6]:
with torch.no_grad():
  sum_loss = 0
  model.eval()
  for bi, sample in enumerate(test_dataloader):
    X = torch.Tensor(sample["list"])
    X = X.to(torch.float32) 
    y = torch.Tensor(sample["mat"])
    y = y.to(torch.float32)
    y_pred = model(X)           # compute model output
    if ei == 50 and (bi % print_freq == 0):
      pass     
    loss = loss_fn(y, y_pred)  # calculate loss
    sum_loss += loss
    if bi % (print_freq*10) == 0: 
      print(f"batch {bi} loss {loss} sum_loss {sum_loss}")

batch 0 loss 0.09204287081956863 sum_loss 0.09204287081956863
batch 200 loss 0.08954490721225739 sum_loss 18.215654373168945
batch 400 loss 0.10235906392335892 sum_loss 36.236358642578125
batch 600 loss 0.08598863333463669 sum_loss 54.36198043823242
batch 800 loss 0.08811386674642563 sum_loss 72.5230484008789
batch 1000 loss 0.09514040499925613 sum_loss 90.70939636230469
batch 1200 loss 0.091006800532341 sum_loss 108.84637451171875
batch 1400 loss 0.0850461795926094 sum_loss 126.89704132080078
batch 1600 loss 0.09272421896457672 sum_loss 145.00184631347656
batch 1800 loss 0.08763337880373001 sum_loss 163.06765747070312
batch 2000 loss 0.09209877252578735 sum_loss 181.2366180419922


In [7]:
test = torch.Tensor([4, 3, 2, 1])
test = test.unsqueeze(0)
sample = test_dataset[0]
X = sample["list"]
X = X.unsqueeze(0)
res = model(X)
print(X)
print(res)

tensor([[6., 2., 5., 7., 0., 1., 3., 4.]])
tensor([[[0.2797, 0.1029, 0.1029, 0.1029, 0.1029, 0.1029, 0.1029, 0.1029],
         [0.1028, 0.2795, 0.1028, 0.1034, 0.1028, 0.1028, 0.1028, 0.1028],
         [0.1030, 0.1030, 0.1030, 0.2783, 0.1040, 0.1030, 0.1030, 0.1030],
         [0.1024, 0.1024, 0.1024, 0.1024, 0.2783, 0.1073, 0.1024, 0.1024],
         [0.1024, 0.1024, 0.1024, 0.1024, 0.1024, 0.2785, 0.1069, 0.1024],
         [0.0991, 0.0991, 0.1362, 0.0991, 0.0991, 0.0991, 0.2693, 0.0991],
         [0.1010, 0.1010, 0.2741, 0.1200, 0.1010, 0.1010, 0.1010, 0.1010],
         [0.1026, 0.1026, 0.1027, 0.2788, 0.1054, 0.1026, 0.1026, 0.1026],
         [0.0866, 0.0868, 0.0866, 0.0984, 0.2334, 0.2349, 0.0866, 0.0866],
         [0.0824, 0.1417, 0.0824, 0.0824, 0.2238, 0.2226, 0.0824, 0.0824],
         [0.0874, 0.2376, 0.2376, 0.0875, 0.0878, 0.0874, 0.0874, 0.0874],
         [0.1631, 0.1640, 0.1640, 0.1640, 0.1640, 0.0603, 0.0603, 0.0603],
         [0.1638, 0.1639, 0.1639, 0.1639, 0.1637, 0.0603,

In [21]:
torch.sum(res, dim=2)
torch.argmax(res, dim=2)


tensor([[0, 1, 3, 4, 5, 6, 2, 3, 5, 4, 2, 2, 1, 1, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7,
         7, 7, 7, 7]])

In [11]:
class Square(nn.Module):
    """ Custom Linear layer but mimics a standard linear layer """
    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        weights = torch.Tensor(size_out, size_in)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
        bias = torch.Tensor(size_out)
        self.bias = nn.Parameter(bias)

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)
        bound = 1 / math.sqrt(fan_in)
        nn.init.uniform_(self.bias, -bound, bound)  # bias init

    def forward(self, x):
        w_times_x= torch.mm(x, self.weights.t())
        # 
        print(f" x shape {x.shape} weights shape {self.weights.shape} res shape {w_timex_x.shape}")
        return torch.add(w_times_x, self.bias) 